# 中文虚假新闻数据集探索分析

本notebook对中文虚假新闻数据集进行探索性分析，包括：
1. 数据加载与预处理
2. 数据基本统计分析
3. 文本特征分析
4. 用户特征分析
5. 传播特征分析
6. 数据可视化

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import json
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import jieba
from collections import Counter
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # 对于macOS
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

## 1. 数据加载

我们将加载两个数据集：
1. `rumors_v170613.json`: 包含31669条谣言原微博数据
2. `CED_Dataset`: 包含谣言和非谣言数据，以及对应的转发信息

In [ ]:
# 定义数据路径
DATA_DIR = Path('../data')
RUMORS_FILE = DATA_DIR / 'rumors_v170613.json'
CED_DATASET = DATA_DIR / 'CED_Dataset'

# 加载原始谣言数据
def load_rumors():
    rumors = []
    with open(RUMORS_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            rumors.append(json.loads(line))
    return pd.DataFrame(rumors)

# 加载CED数据集
def load_ced_dataset():
    # 加载原始微博
    original_path = CED_DATASET / 'original-microblog'
    originals = []
    for file in tqdm(list(original_path.glob('*.json')), desc='Loading original posts'):
        with open(file, 'r', encoding='utf-8') as f:
            originals.append(json.load(f))
    
    # 加载谣言转发
    rumor_path = CED_DATASET / 'rumor-repost'
    rumors = []
    for file in tqdm(list(rumor_path.glob('*.json')), desc='Loading rumor reposts'):
        with open(file, 'r', encoding='utf-8') as f:
            rumors.append(json.load(f))
            
    # 加载非谣言转发
    non_rumor_path = CED_DATASET / 'non-rumor-repost'
    non_rumors = []
    for file in tqdm(list(non_rumor_path.glob('*.json')), desc='Loading non-rumor reposts'):
        with open(file, 'r', encoding='utf-8') as f:
            non_rumors.append(json.load(f))
            
    return pd.DataFrame(originals), pd.DataFrame(rumors), pd.DataFrame(non_rumors)

# 加载数据
print("加载原始谣言数据...")
rumors_df = load_rumors()
print("\n加载CED数据集...")
original_df, rumor_reposts_df, non_rumor_reposts_df = load_ced_dataset()

print("\n数据集大小：")
print(f"原始谣言数据：{len(rumors_df)}条")
print(f"CED原始微博：{len(original_df)}条")
print(f"CED谣言转发：{len(rumor_reposts_df)}条")
print(f"CED非谣言转发：{len(non_rumor_reposts_df)}条")

## 2. 数据基本统计分析

让我们首先对数据进行一些基本的统计分析：
1. 查看数据的基本信息和字段分布
2. 分析谣言和非谣言的时间分布
3. 分析用户特征的统计信息
4. 分析转发行为的统计特征

In [ ]:
# 2.1 查看数据基本信息
print("原始谣言数据集信息：")
print(rumors_df.info())
print("\n原始谣言数据样例：")
print(rumors_df.head())

print("\nCED数据集原始微博信息：")
print(original_df.info())
print("\nCED数据集原始微博样例：")
print(original_df.head())

# 2.2 时间分布分析
# 将时间字符串转换为datetime对象
rumors_df['publishTime'] = pd.to_datetime(rumors_df['publishTime'])

# 按年份统计谣言数量
yearly_counts = rumors_df['publishTime'].dt.year.value_counts().sort_index()

plt.figure(figsize=(12, 6))
yearly_counts.plot(kind='bar')
plt.title('谣言年度分布')
plt.xlabel('年份')
plt.ylabel('谣言数量')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2.3 用户特征分析
# 统计发布谣言最多的用户
top_rumormongers = rumors_df['rumormongerName'].value_counts().head(10)

plt.figure(figsize=(12, 6))
top_rumormongers.plot(kind='bar')
plt.title('发布谣言最多的用户Top10')
plt.xlabel('用户名')
plt.ylabel('发布谣言数量')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2.4 转发特征分析
def count_reposts(row):
    if isinstance(row, list):
        return len(row)
    return 0

rumor_reposts_df['repost_count'] = rumor_reposts_df['reposts'].apply(count_reposts)
non_rumor_reposts_df['repost_count'] = non_rumor_reposts_df['reposts'].apply(count_reposts)

plt.figure(figsize=(10, 6))
plt.hist([rumor_reposts_df['repost_count'], non_rumor_reposts_df['repost_count']], 
         label=['谣言', '非谣言'], bins=30, alpha=0.7)
plt.title('谣言与非谣言转发数量分布')
plt.xlabel('转发数量')
plt.ylabel('频率')
plt.legend()
plt.tight_layout()
plt.show()

## 3. 文本特征分析

接下来我们将分析文本内容的特征：
1. 文本长度分布
2. 高频词统计
3. 词云可视化
4. 关键词特征分析

In [ ]:
# 3.1 文本长度分布分析
rumors_df['text_length'] = rumors_df['rumorText'].str.len()
original_df['text_length'] = original_df['text'].str.len()

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(rumors_df['text_length'], bins=50)
plt.title('谣言文本长度分布')
plt.xlabel('文本长度')
plt.ylabel('频率')

plt.subplot(1, 2, 2)
plt.hist(original_df['text_length'], bins=50)
plt.title('CED数据集文本长度分布')
plt.xlabel('文本长度')
plt.ylabel('频率')
plt.tight_layout()
plt.show()

# 3.2 高频词统计
def get_word_freq(texts, top_n=20):
    # 加载停用词
    stop_words = set(['的', '了', '是', '在', '我', '有', '和', '就', '不', '人', '都', '一', '一个', '上', '也', '很', '到', '说', '要', '去', '你', '会', '着', '没有', '看', '好', '自己', '这'])
    
    # 分词并统计词频
    word_freq = Counter()
    for text in tqdm(texts, desc='Processing texts'):
        if isinstance(text, str):
            words = jieba.lcut(text)
            words = [w for w in words if w not in stop_words and len(w) > 1]
            word_freq.update(words)
    
    return pd.DataFrame(word_freq.most_common(top_n), columns=['word', 'frequency'])

# 分析谣言文本的高频词
rumor_words = get_word_freq(rumors_df['rumorText'])
print("\n谣言文本高频词：")
print(rumor_words)

plt.figure(figsize=(12, 6))
plt.bar(rumor_words['word'], rumor_words['frequency'])
plt.title('谣言文本高频词统计')
plt.xlabel('词语')
plt.ylabel('频率')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 3.3 对比分析CED数据集中谣言和非谣言的词频差异
rumor_texts = original_df[original_df['label'] == 1]['text']
non_rumor_texts = original_df[original_df['label'] == 0]['text']

rumor_words = get_word_freq(rumor_texts)
non_rumor_words = get_word_freq(non_rumor_texts)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.bar(rumor_words['word'], rumor_words['frequency'])
plt.title('CED数据集谣言高频词')
plt.xlabel('词语')
plt.ylabel('频率')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.bar(non_rumor_words['word'], non_rumor_words['frequency'])
plt.title('CED数据集非谣言高频词')
plt.xlabel('词语')
plt.ylabel('频率')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. 数据分析总结

通过以上分析，我们可以得出以下结论：

1. **数据集规模**
   - 原始谣言数据集包含大量样本，适合深度学习模型训练
   - CED数据集包含谣言和非谣言样本，适合二分类任务

2. **文本特征**
   - 谣言文本普遍较长，包含更多细节描述
   - 谣言和非谣言在用词上存在显著差异
   - 某些特定词语在谣言中出现频率更高

3. **传播特征**
   - 谣言的转发量普遍高于非谣言
   - 谣言的传播速度和范围都较大

4. **时间特征**
   - 谣言的发布时间呈现出明显的周期性
   - 某些时间段谣言传播更为频繁

## 下一步工作

1. 基于分析结果设计特征工程方案
2. 构建文本分类模型
3. 加入传播特征进行模型优化
4. 设计评估实验